# <b>Preparación del entorno para la demo</b>

Esta demo requiere que tengais instalado de antes la aplicación *docker.desktop* y que tengais activado en la aplicación un cluster de kubernetes, todos los comandos son ejecutables desde terminal y para ver los resultados completos recomendamos el uso de al menos 2 terminales, una para ejecutar comandos y la otra para monitorear las respuestas.

La demo incluye:

    -Una app sencilla que se encarga de devolver el nombre del host (para poder diferenciar a los contenedores/pods) y las funciones de testeo stress_cpu y crash
    -un archivo deplyment-stress.yaml y service-stress.yaml que sirven para configurar kubernetes
    -un archivo dokerfile de configuración de contenedores
    -un archivo requirements.txt para facilitar la instalación de dependencias


Aqui están los pasos a  seguir para preparar el entorno de la demo y comprobar que todo funciona correctamente:

In [ ]:
#Crear entorno virtual
python -m venv .venv

source .venv/bin/activate

#instalar dependencias
pip install -r requirements.txt

#comprobar que la app funciona con el python del entorno virtual
python -m pip --version
python app/app.py

#Comprobar que funcionan los endpoints de la página
curl http://localhost:5000/
curl http://localhost:5000/stress-cpu

#Crear dockerfile

#Construir una imagen y comprobar que aparece en la lista de imagenes
docker build -t stress-app:latest .
docker images | grep stress-app

# <b>Limpieza de la demo</b>

Cuando querais repetir las pruebas de la demo o acabes de preparar el entorno, podeis usar estos comandos para borrar los contenedores Docker y los recursos de Kubernetes para que tengais un entorno limpio con el que empezar.

In [ ]:
#!/usr/bin/env bash

#Recursos de kubernetes
kubectl delete deployment stress-app --ignore-not-found=true
kubectl delete svc stress-app-svc --ignore-not-found=true
kubectl delete hpa stress-app --ignore-not-found=true

#Contenedores Docker
docker rm -f stress-free 2>/dev/null || true
docker rm -f stress-limited 2>/dev/null || true

#Comprobamos que está limpio
kubectl get all

# <b> 1. Contenedores Docker</b>



Docker nos permite empaquetar aplicaciones y sus dependencias dentro de lo que llamamos *contenedores*

Estos *contenedores* son procesos aislados, con su propio sistema de archivos y que puede ejecutarse igual en cualquier máquina.

Hay que saber que no tiene kernel propio ni es una máquina virtual, es un proceso empaquetado y muy ligero.

## <b>Dockerfile</b>

Para describir como construir una imagen Docker, utilizamos el *dockerfile* que tiene esta forma:

    (ejemplo básico)
    FROM python:3.10-slim                     <- Imagen a usar (esta es de Python)
    WORKDIR /app                              <- Carpeta donde se ejecutan comandos
    COPY requirements.txt .                   <- copia un archivo al contenedor
    RUN pip install -r requirements.txt       <- instala las dependencias  
    COPY . .                                  <- Copia todo el proyecto en el contenedor
    CMD ["python3", "app.py"]                 <- Comando por defecto de arranque

Con esto vamos a crear una imagen de Docker y vamos a hacer tests de prueba para ver su flexibilidad y resistencia

In [ ]:

#Primero construimos la imagen y paramos y eliminamos cualquier contenedor previo si existe
docker build -t stress-app:latest .
docker rm -f stress-free 2>/dev/null || true

#Lanzamos  la imagen
docker run -d --name stress-free -p 8080:5000 stress-app:latest

Una vez levantado, podemos ver los contenedores que están funcionando con el siguiente comando por terminal.

In [ ]:
#Muestra todos los contenedores, su id, imagen, estado, pueto y nombre
docker ps

#Si queremos ver todos los contenedores levantados y 
docker stats

#Si queremos ver un contenedor en concreto 
docker stats stress-free

Cuando esté todo preparado, podemos ver como se sobrecarga la CPU del contenedor al ejecutar el siguiente comando en una terminal, y viendo los stats de Docker en otra

In [ ]:
#Este primer curl utiliza el endpoint basico para comprobar la conexión y recibir información básica del contenedor
curl http://localhost:8080/

#Este inicia el test de estrés
curl http://localhost:8080/stress-cpu

En el momento de realizar el test, el consumo de CPU se dispara al 100% y no puede aceptar más carga de trabajo hasta que termina la prueba y responde con un mensaje con el siguiente formato.
    
    {"iterations":72903670,"message":"CPU estresada durante ~10 segundos","status":"ok"}

# <b> 2. Limitar Contenedores 

Los contenedores no siempre utilizarán la misma configuración, podemos limitar los recursos disponibles para un contenedor para simular condiciones reales de producción, evitar que un proceso consuma toda la CPU o gestionar varios servicios en la misma máquina

    --memory=""         <- Limita la cantidad de memoria RAM
    --cpus=""           <- Define cuantos núcleos de CPU se pueden usar
    --pids-limit=       <- Define el número máximo de procesos que se pueden ejecutar simultaneamente
    --memory-swap       <- Establece el limite total de memoria (RAM + Swap)
    -p 8080:5000        <- Mapea un puerto del host (8080) a un puerto interno del contenedor (5000)

Ahora vamos a hacer que nuestro contenedor no use el 100% de la RAM, limitar su uso de CPU al 50% y en vez de usar el puerto 8080 usaremos el 8081.

In [ ]:
docker run -d --name stress-limited --memory="256m" --cpus="0.5" -p 8081:5000 stress-app:latest


Para ver de nuevo los resultados, abrimos dos terminales, una con Docker Stats y en la otra iniciamos las pruebas.

In [ ]:
docker stats stress-limited

curl http://localhost:8081/stress-cpu

Ahora la carga sobre la CPU se limita al 50% y el contenedor no ocupa 16Gb de memoria, podemos ver que el número de iteraciones se reduce a la mitad asi que el contenedor se está "colgando" mucho más rápido 

    *{"iterations":38128045,"message":"CPU estresada durante ~10 segundos","status":"ok"}*

¿Qué pasaría si el contenedor sufre un fallo crítico y no una carga excesiva de trabajo?

In [ ]:
curl http://localhost:8081/crash

El contenedor se frena inesperadamente y no se reinicia. Docker ofrece un aislamiento de recursos, pero la recuperación de los contenedores es manual.

# <b> 3. Kubernetes y auto-recuperación de Contenedores

Kubernetes es un orquestador de contenedores, nos proporciona *autorecuperación*, *escalado*, *reparto de carga*, *aislamiento de servicios* y en esencia, mantiene el sistema en el estado deseado.

Al igual que en Docker teníamos el *dockerfile*, con kubernetes también usamos unos manifiestos con esta estructura:

    apiVersion:    <- Versión de la API que se está usando
    kind:          <- Tipo de objeto (Deployment, Service, Pod…)
    metadata:      <- nombre, etiquetas
    spec:          <- especificación: lo que quieres que exista 

Por ejemplo en el *deployment-stress.yaml* vemos que usamos un *Deployment con la *metadata* de name y lables y en *spec* definimos cuantas réplicas usamos, como van a ser los contenedores y los recursos a usar, siguiendo la estructura de nuestro anterior dockerfile y configuración de recursos.

El *deployment* es un objeto de alto nivel y define cuantas replicas debe haber, al aplicarlo se crean también:

-*ReplicaSet* que controla el número de pods y si muere uno, crea otro.

-Pod, un contenedor ejecutandose (unidad mínima de kubernetes)

El *service* es la forma de exponer los pods al exterior, usamos el *NodePort* que significa que es accesible desde fuera del cluster. *ClusterIP* haría que solo sea accesible desde dentro del cluster. 

In [ ]:
#Aplicamos los dos archivos .yaml para configurar nuestro cluster
kubectl apply -f k8s/deployment-stress.yaml
kubectl apply -f k8s/service-stress.yaml

Una vez se hayan aplicado los .yaml podemos ver todos nuestros deployments, nuestros pods y el svc.

In [ ]:
kubectl get deployments
kubectl get pods
kubectl get svc

El comando *"kubectl get pods"* nos permite ver si un pod está corriendo, cuantas veces se ha reiniciado y el tiempo que lleva levantado, lo usaremos para  ver como reacciona cuando producimos un fallo crítico. 

In [ ]:

#En una terminal a parte para ver como reacciona el pod
kubectl get pods -w

#Crash
curl http://localhost:30080/crash

El pod pasa de estar Running a Error, en el momento en el que esto ocurre kubernetes se encarga de recrear el pod automaticamente.

    NAME                          READY   STATUS    RESTARTS   AGE
    stress-app-6bd7d969b5-qf7pq   1/1     Running   0          3m7s
    stress-app-6bd7d969b5-qf7pq   0/1     Error     0          5m50s
    stress-app-6bd7d969b5-qf7pq   0/1     Running   1 (2s ago)   5m51s
    stress-app-6bd7d969b5-qf7pq   1/1     Running   1 (8s ago)   5m57s

Este reinicio automático es gracias a la configuración del *deplyoment* donde usamos *Liveness probe*

    spec:
        containers:
            livenessProbe:
                    httpGet:
                    path: /
                    port: 5000
                    initialDelaySeconds: 15
                    periodSeconds: 10
                    timeoutSeconds: 2
                    failureThreshold: 3

Indica si el pod está vivo, si falla kubernetes mata el contenedor y el ReplicaSet crea otro. Asi kubernetes gestiona fallos críticos y restaura el estado original del contenedor.

# <b> 4. Reparto de carga de trabajo con Kubernetes

Hemos visto como Docker puede limitar los recursos de un contenedor y como kubernetes ofrece resistencia ante fallos y reparación automática, pero en el momento que falla el contenedor y se está recreando se para la conexión hasta que vuelva a estar *READY* y *RUNNING*, asi que podemos crear varias réplicas de este pod para que aunque falle una, las demás sigan funcionando sin problema alguno.

El *service-stress.yaml* indica como kubernetes reparte la carga entre distintos pods gracias al selector:

    spec:
        selector:
            app: stress-app
        type: NodePort

Se encarga de crear un endpoint con todos los pods candidatos y cada request se envia aleatoriamente a uno de ellos, ¿pero que pasa si un pod candidato falla?

Esto lo conseguimos solucionar gracias a la configuración del *deployment* con *readinessProbe*:

    spec:
        containers:
            readinessProbe:
                 httpGet:
                    path: /
                    port: 5000
                initialDelaySeconds: 5
                periodSeconds: 5
                timeoutSeconds: 2
                failureThreshold: 3

Indica si un pod está listo para recibir tráfico, si falla, el pod sigue vivo pero no recibe peticiones.


In [ ]:
#Escalamos el deployment de nuestra app X numero de veces
kubectl scale deployment stress-app --replicas=3

#Con este comando podemos ir viendo como se van activando las réplicas
kubectl rollout status deployment/stress-app

#Si tenemos la lista de pods abierta podemos ver como se crean todas las réplicas de la misma app pero distinta id
kubectl get pods -w

Para comprobar que la carga de trabajo se reparte entre todas las réplicas, podemos pedirle al endpoint basico de la app que nos devuelva constantemente el nombre del pod que lo está ejecutando.

In [ ]:
#Pedimos 20 veces el nombre del host
for i in {1..20}; 
do
    curl -s http://localhost:30080/ | grep hostname
    sleep 1
done

Ahora vamos a elegir un pod de los que tenemos y le mandaremos comandos directamente a el para hacer que se rompa, asi comprobamos si se retá redirigiendo correctamente el tráfico hacia los otros  pods.

In [ ]:
#Elegimos un pod y comprobamos que le hablamos a el
kubectl port-forward pod/ <nombrePod> 5001:5000

curl http://localhost:5001


#Ahora lo tiramos y comprobamos que el servicio sigue funcionando
curl http://localhost:5001/crash

El pod elegido se cuelga y no recibe peticiones, pero los otros siguen funcionando sin problemas y en el momento que el pod vuelve a estar disponible puede recibir las peticiones de nuevo.

    stress-app-6bd7d969b5-4rlgz   1/1     Running   1 (8s ago)    11m
    stress-app-6bd7d969b5-4rlgz   0/1     Error     1 (55s ago)   11m
    stress-app-6bd7d969b5-4rlgz   0/1     CrashLoopBackOff   1 (4s ago)    11m
    stress-app-6bd7d969b5-4rlgz   0/1     Running            2 (11s ago)   12m

Aunque la réplica muere, kubernetes se encarga de que el servicio siga respondiendo y repone la instancia fallida.